In [155]:
import pandas as pd

In [156]:
#class, age, fare별 생존률 구하는데 쓰임
train = pd.read_csv('train.csv')
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [157]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [158]:
train.shape  #행, 열 개수

(891, 12)

In [159]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [160]:
train.groupby('Survived').size()  #train.value_counts('Survived')

Survived
0    549
1    342
dtype: int64

In [161]:
train.value_counts('Survived', normalize=True)  #비율***

Survived
0    0.616162
1    0.383838
Name: proportion, dtype: float64

'성별'별 생존율

In [162]:
total = train.groupby('Sex').size().to_frame('전체인원')
total


,전체인원
Sex,
female,314
male,577


In [163]:
filt = train['Survived'] == 1
survived = train[filt]
survived = survived.groupby('Sex').size().to_frame('생존인원')
survived

,생존인원
Sex,
female,233
male,109


In [164]:
gender = total.join(survived)  #total.merge(survived, on='Sex')
gender['생존율'] = gender['생존인원']/gender['전체인원']
gender  #여자가 생존율이 더 높다

,전체인원,생존인원,생존율
Sex,,,
female,314,233,0.742038
male,577,109,0.188908


'티켓 클래스'별 생존율

In [165]:
total = train.groupby('Pclass').size().to_frame('전체인원')
total

,전체인원
Pclass,
1,216
2,184
3,491


In [166]:
filt = train['Survived'] == 1
survived = train[filt]
pclass = survived.groupby('Pclass').size().to_frame('생존인원')
pclass

,생존인원
Pclass,
1,136
2,87
3,119


In [167]:
pclass = total.join(pclass)
pclass['생존율'] = pclass['생존인원']/pclass['전체인원']
pclass  #class가 좋을 수록 생존율이 높다

,전체인원,생존인원,생존율
Pclass,,,
1,216,136,0.629630
2,184,87,0.472826
3,491,119,0.242363


'연령대'별 생존율

In [168]:
train['Age'].describe()

count    714.000000
mean      29.699118
std       14.526497
min        0.420000
25%       20.125000
50%       28.000000
75%       38.000000
max       80.000000
Name: Age, dtype: float64

In [169]:
filt = train['Age'] == train['Age'].min()
train[filt]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
803,804,1,3,"Thomas, Master. Assad Alexander",male,0.42,0,1,2625,8.5167,NaN,C


In [170]:
ages = []
min = int(train['Age'].min())
max = int(train['Age'].max())

step = 20
for age in range(min, max+1, step):  #min에서 max+1까지 10씩 증가
    ran = f'{age}이상 ~ {age+step}미만'

    #연령대별 인원수
    filt = (train['Age'] >= age) & (train['Age'] < age+step)
    total = train[filt]
    count = train[filt]['Age'].count()  

    #연령대별 생존자수
    filt = (train['Age'] >= age) & (train['Age'] < age+step) & (train['Survived']==1)
    count_survived = train[filt]['Age'].count()  
    ages.append({
        '나이': ran, 
        '전체인원':count,
        '생존인원':count_survived
    })

    df = pd.DataFrame(ages)
    df.set_index('나이', inplace=True)
    df['생존율'] = df['생존인원']/df['전체인원']
    df.sort_values('생존율', ascending=False)

In [171]:
df

,전체인원,생존인원,생존율
나이,,,
0이상 ~ 20미만,164,79,0.481707
20이상 ~ 40미만,387,150,0.387597
40이상 ~ 60미만,137,54,0.394161
60이상 ~ 80미만,25,6,0.240000
80이상 ~ 100미만,1,1,1.000000


In [172]:
df['전체인원'].sum()

np.int64(714)

In [173]:
#탑승항구Embarked별 전체인원
em = train.groupby('Embarked').size().to_frame('전체인원')
em

,전체인원
Embarked,
C,168
Q,77
S,644


In [174]:
#생존인원
filt = train['Survived'] == 1
survived = train[filt]
survived = survived.groupby('Embarked').size().to_frame('생존인원')
survived

,생존인원
Embarked,
C,93
Q,30
S,217


In [175]:
join = em.join(survived)
join

,전체인원,생존인원
Embarked,,
C,168,93
Q,77,30
S,644,217


In [176]:
join['사망률'] = 1 - join['생존인원']/join['전체인원']

In [177]:
join

,전체인원,생존인원,사망률
Embarked,,,
C,168,93,0.446429
Q,77,30,0.610390
S,644,217,0.663043


In [178]:
group = train.groupby(['Embarked', 'Pclass']).size()
c = group['C']
s = group['S']
q = group['Q']

df = pd.DataFrame([c, s, q], index=['C', 'S', 'Q'])
df.columns = ['1등석', '2등석', '3등석']
df.index.name = '선박장'
df

,1등석,2등석,3등석
선박장,,,
C,85,17,66
S,127,164,353
Q,2,3,72


In [179]:
group = train.groupby(['Pclass', 'Embarked']).size()
a = group[1]
b = group[2]
c = group[3]

df = pd.DataFrame([a, b, c], index=['1등석', '2등석', '3등석'])
df.index.name = '티켓 클래스'

df

Embarked,C,Q,S
티켓 클래스,,,
1등석,85,2,127
2등석,17,3,164
3등석,66,72,353


In [203]:
group = train.groupby(['Embarked', 'Pclass']).size()

df = pd.DataFrame()
for idx in group.index:
    df.loc[idx[0], idx[1]]= int(group.loc[idx])

df.index.name='항구'
cols = df.columns
df.columns = [str(col)+'등석' for col in cols]

df

,1등석,2등석,3등석
항구,,,
C,85.0,17.0,66.0
Q,2.0,3.0,72.0
S,127.0,164.0,353.0
